# Language Warmup Full Model

## Data Cleaning

## Feature Engineering and Vectorization

In [2]:
import random

with open("FeatureCreate\words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]

phrases = []
for i in range (0,10):
    phraselength = random.randint(5,15)
    phrase = []
    for j in range(0,phraselength):
        choice = random.randint(0,len(words))
        phrase.append(words[choice])
        sentence = ' '.join(phrase)
    phrases.append(sentence)

# REMOVE ABOVE

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, lowercase=False, ngram_range=(2, 2))
vector = vectorizer.fit_transform(phrases)


In [10]:
# Make fake y values

for i in range 

In [9]:
data = vector.todense()


## Model Architecture

In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1,  activation = 'sigmoid'))

Using TensorFlow backend.


In [6]:
model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])